**THIS ONE BELOW IS ONLY FOR EQUALITY TYPE CONSTRAINTS, THE CODE BELOW THAT SUPPORTS ALL TYPE OF EQUALITY AND INEQUALITIES**

In [2]:
import numpy as np

def update_zj_cj_two_phase(new_A, new_B, cb_matrix, n_variables, n_constraints, _type):
    tableau_without_rhs = [row[:] for row in new_A]
    transpose_A = list(map(list, zip(*tableau_without_rhs)))
    limit = n_variables if _type == '2' else n_variables + n_constraints
    zj = [np.dot(cb_matrix, transpose_A[j]) for j in range(limit)]
    cj = [-new_B[i] + zj[i] for i in range(limit)]   
    return zj, cj


In [4]:
def two_phase_simplex_method(objective_fun, A, B, type):
    n_constraints = len(A)
    n_variables = len(objective_fun)
    cb = [-1] * n_constraints
    basic = list(range(n_variables, n_variables + n_constraints))
    new_A = []
    new_B = [0] * n_variables + [-1] * n_constraints
    for i, row in enumerate(A):
        new_row = row.copy()
        identity_part = [1 if j == i else 0 for j in range(n_constraints)]
        new_row.extend(identity_part)
        new_row.append(B[i])
        new_A.append(new_row)
    print("Initial tableau:")
    for row in new_A:
        print(row)
    zj,cj = update_zj_cj_two_phase(new_A,new_B,cb,n_variables,n_constraints,'1')
    print(f"\nInitial Cj: {cj}")
    print(f"Initial basic variables: {basic}")
    print(f"Initial cb_matrix: {cb}")
    print(f"Initial zj: {zj}")
    iteration = 0
    max_iterations = 100
    if type.lower() == 'maximize':
        print('-'*50)
        while(min(cj)< 0  and iteration < max_iterations):
            iteration +=1
            print(f"\n--- Iteration {iteration} ---")
            key_column = np.argmin(cj)
            possible_rows = [i for i in range(n_constraints) if new_A[i][key_column] > 0]
            if not possible_rows:
                print("Problem is unbounded.")
                return None
            ratios = [abs(new_A[i][-1] / new_A[i][key_column]) for i in possible_rows]
            print(f"Possible rows: {possible_rows}")
            print(f"Ratios: {ratios}")
            min_ratio_idx = np.argmin(ratios)
            key_row = possible_rows[min_ratio_idx]
            print(f"Leaving variable row (key row): {key_row}")
            pivot_element = new_A[key_row][key_column]
            print(f"Pivot element: {pivot_element}")
            if abs(pivot_element) < 1e-10:
                print("Division by zero encountered (pivot is zero).")
                return None
            for j in range(len(new_A[key_row])):
                new_A[key_row][j] /= pivot_element
            for i in range(n_constraints):
                if i != key_row:
                    factor = new_A[i][key_column]
                    for j in range(len(new_A[i])):
                        new_A[i][j] -= factor * new_A[key_row][j]
            old_basic = basic[key_row]
            basic[key_row] = key_column
            cb[key_row] = new_B[key_column]
            print(f"Variable {old_basic} leaves, variable {key_column} enters")
            print(f"Updated basic variables: {basic}")
            print(f"Updated cb_matrix: {cb}")
            zj, cj = update_zj_cj_two_phase(new_A,new_B,cb,n_variables,n_constraints,'1')
            print(f"Updated Cj: {[round(c, 6) for c in cj]}")
            print("Current tableau:")
            for i, row in enumerate(new_A):
                print(f"Row {i}: {[round(x, 6) for x in row]}")
            print('------')
        new_B = objective_fun
        zj, cj = update_zj_cj_two_phase(new_A,new_B,cb,n_variables,n_constraints,'2')
        print(f"\nInitial ---- Cj: {cj}")
        print(f"Initial basic ----variables: {basic}")
        print(f"Initial cb_=-----: {cb}")
        print(cj)
        entering_phase_2 = False
        while(min(cj)< 0  and iteration < max_iterations):
            entering_phase_2 = True
            print("ENTERING PHASE 2")
            iteration +=1
            print(f"\n--- Iteration {iteration} ---")
            key_column = np.argmin(cj)
            possible_rows = [i for i in range(n_constraints) if new_A[i][key_column] > 0]
            if not possible_rows:
                print("Problem is unbounded.")
                return None

            ratios = [abs(new_A[i][-1] / new_A[i][key_column]) for i in possible_rows]
            print(f"Possible rows: {possible_rows}")
            print(f"Ratios: {ratios}")
            min_ratio_idx = np.argmin(ratios)
            key_row = possible_rows[min_ratio_idx]
            print(f"Leaving variable row (key row): {key_row}")
            pivot_element = new_A[key_row][key_column]
            print(f"Pivot element: {pivot_element}")

            if abs(pivot_element) < 1e-10:
                print("Division by zero encountered (pivot is zero).")
                return None
            for j in range(len(new_A[key_row])):
                new_A[key_row][j] /= pivot_element
            for i in range(n_constraints):
                if i != key_row:
                    factor = new_A[i][key_column]
                    for j in range(len(new_A[i])):
                        new_A[i][j] -= factor * new_A[key_row][j]

            old_basic = basic[key_row]
            basic[key_row] = key_column
            cb[key_row] = new_B[key_column]
            print(f"Variable {old_basic} leaves, variable {key_column} enters")
            print(f"Updated basic variables: {basic}")
            print(f"Updated cb_matrix: {cb}")
            zj, cj = update_zj_cj_two_phase(new_A,new_B,cb,n_variables,n_constraints,'2')
            print(f"Updated Cj: {[round(c, 6) for c in cj]}")
            print("Current tableau:")
            for i, row in enumerate(new_A):
                print(f"Row {i}: {[round(x, 6) for x in row]}")
        if(entering_phase_2):        
         print(f"\n--- Final Solution (after {iteration} iterations) ---")
         x = [0.0] * n_variables
         for i in range(n_constraints):
          col = basic[i]
          if col < n_variables:  
            x[col] = new_A[i][-1]
         bj = [row[-1] for row in new_A]
         optimal_value = np.dot(cb, bj)
         print(f"Basic variables: {basic}")
         print(f"Solution: {[round(xi, 6) for xi in x]}")
         print(f"Optimal value: {round(optimal_value, 6)}")
        print(entering_phase_2) 
    return -1
# objective_fun=[2,3,10]
# A=[[1,0,2],[0,1,1]]
# B=[1,1]

# objective_fun=[45,90,135,145]
# A=[[15,-10,25,50],[125,-50,200,75],[225,80,-150,40],[200,60,30,50]]
# B=[1000,2000,3000,4000]

objective_fun=[500,600,800,2100]
A=[[500,600,700,200],[700,700,800,1700],[50,100,150,700],[450,150,250,250]]
B=[2000,3900,1000,1100]


result = two_phase_simplex_method(objective_fun, A, B, 'maximize')


Initial tableau:
[500, 600, 700, 200, 1, 0, 0, 0, 2000]
[700, 700, 800, 1700, 0, 1, 0, 0, 3900]
[50, 100, 150, 700, 0, 0, 1, 0, 1000]
[450, 150, 250, 250, 0, 0, 0, 1, 1100]

Initial Cj: [np.int64(-1700), np.int64(-1550), np.int64(-1900), np.int64(-2850), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Initial basic variables: [4, 5, 6, 7]
Initial cb_matrix: [-1, -1, -1, -1]
Initial zj: [np.int64(-1700), np.int64(-1550), np.int64(-1900), np.int64(-2850), np.int64(-1), np.int64(-1), np.int64(-1), np.int64(-1)]
--------------------------------------------------

--- Iteration 1 ---
Possible rows: [0, 1, 2, 3]
Ratios: [10.0, 2.2941176470588234, 1.4285714285714286, 4.4]
Leaving variable row (key row): 2
Pivot element: 700
Variable 6 leaves, variable 3 enters
Updated basic variables: [4, 5, np.int64(3), 7]
Updated cb_matrix: [-1, -1, 0, -1]
Updated Cj: [np.float64(-1496.428571), np.float64(-1142.857143), np.float64(-1289.285714), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.floa

**BELOW IMPLEMENTATION INCLUDES SUPPORT FOR ALL TYPES OF EQUALITIES AND INEQUALITIES FOR THE 2 PHASE SIMPLEX METHOD.**

In [2]:
def update_zj_cj_two_phase(new_A, new_B, cb_matrix, n_variables, n_constraints, greater_than_count,less_than_count,equal_to_count):
    transpose_A = list(map(list, zip(*new_A)))
    limit = n_variables + 2 * greater_than_count + less_than_count + equal_to_count
    zj = [np.dot(cb_matrix, transpose_A[j]) for j in range(limit)]
    cj = [-new_B[i] + zj[i] for i in range(limit)]
    return zj, cj
def update_zj_cj_two_phase_2(new_A, new_B, cb_matrix, n_variables, n_constraints, greater_than_count,less_than_count,equal_to_count):
    transpose_A = list(map(list, zip(*new_A)))
    if(greater_than_count==0 and less_than_count==0) :
      limit = n_variables
    else:
      limit = n_variables  +  greater_than_count + less_than_count
    zj = [np.dot(cb_matrix, transpose_A[j]) for j in range(limit)]
    cj = [-new_B[i] + zj[i] for i in range(limit)]
    return zj, cj


In [ ]:
#modifying to include inequality type as well
import numpy as np


def two_phase_simplex_method_generalized(objective_fun, A, B, type,_type):
    n_constraints = len(A)
    n_variables = len(objective_fun)
    cb = [0 if _type[i] == '<=' else -1 for i in range(n_constraints)]
    greater_than_count = sum(1 for i in range(len(_type)) if _type[i] == '>=')
    less_than_count = sum(1 for i in range(len(_type)) if _type[i] == '<=')
    equal_to_count = sum(1 for i in range(len(_type)) if _type[i] == '=')
    basic = list(range(n_variables, n_variables + n_constraints))
    new_A = []
    new_B = [0] * n_variables + [0] * (greater_than_count + less_than_count) + [-1] * (greater_than_count + equal_to_count)
    print(new_B)
    for i, row in enumerate(A):
        new_row = row.copy()
        for j in range(greater_than_count + less_than_count):
            if j == i:
                if _type[i] == '<=':
                    new_row.append(1)
                elif _type[i] == '>=':
                    new_row.append(-1)
                else:  # '='
                    pass
            else:
                if _type[j] != '=':
                    new_row.append(0)
        for j in range(greater_than_count + less_than_count):
            if j == i and _type[i] in ['>=', '=']:
                new_row.append(1)
            elif _type[j] in ['>=', '=']:
                new_row.append(0)
        if(greater_than_count==0 and less_than_count==0):
           identity_part = [1 if j == i else 0 for j in range(n_constraints)]
           new_row.extend(identity_part)
        elif(equal_to_count > 0):
          identity_part = [1 if j==i and _type[i] in ['='] else 0 for j in range(n_constraints)]
          new_row.extend([identity_part[-1]])
        new_row.append(B[i])
        new_A.append(new_row)
    print("Initial tableau:")
    for row in new_A:
        print(row)
    zj,cj = update_zj_cj_two_phase(new_A,new_B,cb,n_variables,n_constraints,greater_than_count,less_than_count,equal_to_count)
    print(f"\nInitial Cj: {cj}")
    print(f"Initial basic variables: {basic}")
    print(f"Initial cb_matrix: {cb}")
    print(f"Initial zj: {zj}")
    iteration = 0
    max_iterations = 100
    if type.lower() == 'maximize':
        print('-'*50)
        while(min(cj)< 0  and iteration < max_iterations):
            iteration +=1
            print(f"\n--- Iteration {iteration} ---")
            key_column = np.argmin(cj)
            possible_rows = [i for i in range(n_constraints) if new_A[i][key_column] > 0]
            if not possible_rows:
                print("Problem is unbounded.")
                return None
            ratios = np.array([new_A[i][-1] / new_A[i][key_column] for i in possible_rows])
            ratios = np.where(ratios > 0, ratios, np.inf)   # only strictly positive
            min_ratio_idx = np.argmin(ratios)
            key_row = possible_rows[min_ratio_idx]
            print(f"Possible rows: {possible_rows}")
            print(f"Ratios: {ratios}")
            print(f"Leaving variable row (key row): {key_row}")
            pivot_element = new_A[key_row][key_column]
            print(f"Pivot element: {pivot_element}")
            if abs(pivot_element) < 1e-10:
                print("Division by zero encountered (pivot is zero).")
                return None
            for j in range(len(new_A[key_row])):
                new_A[key_row][j] /= pivot_element
            for i in range(n_constraints):
                if i != key_row:
                    factor = new_A[i][key_column]
                    for j in range(len(new_A[i])):
                        new_A[i][j] -= factor * new_A[key_row][j]
            old_basic = basic[key_row]
            basic[key_row] = key_column
            cb[key_row] = new_B[key_column]
            print(f"Variable {old_basic} leaves, variable {key_column} enters")
            print(f"Updated basic variables: {basic}")
            print(f"Updated cb_matrix: {cb}")
            zj, cj = update_zj_cj_two_phase(new_A,new_B,cb,n_variables,n_constraints,greater_than_count,less_than_count,equal_to_count)
            print(f"Updated Cj: {[round(c, 6) for c in cj]}")
            print("Current tableau:")
            for i, row in enumerate(new_A):
                print(f"Row {i}: {[round(x, 6) for x in row]}")
        new_B = objective_fun +[0]*(greater_than_count+less_than_count)
        print(new_B)
        zj, cj = update_zj_cj_two_phase_2(new_A,new_B,cb,n_variables,n_constraints,greater_than_count,less_than_count,equal_to_count)
        print(f"\n Cj Updated: {cj}")
        print(f"updated basic variables: {basic}")
        print(f"Updated cb =: {cb}")
        entering_phase_2 = False
        xb=[new_A[i][-1] for i in range(len(new_A))]
        if(-1 in cb and np.dot(cb,xb)==0):
          cb1=cb.copy()
          for i,j in enumerate(cb1):
           if j==0:
             cb[i]=objective_fun[basic[i]]
           else:
             cb[i]= 0
        elif(-1 not in cb and np.dot(cb,xb)==0):
          entering_phase_2 = True
        elif(-1 in cb and np.dot(cb,xb)<0):
          print("SOLUTION NOT POSSIBLE ")
          return -1
        while((min(cj)< 0  and iteration < max_iterations)):
            entering_phase_2 = True
            print("ENTERING PHASE 2")
            iteration +=1
            print(f"\n--- Iteration {iteration} ---")
            key_column = np.argmin(cj)
            print(key_column, ' key_column')
            possible_rows = [i for i in range(n_constraints) if new_A[i][key_column] > 0]
            if not possible_rows:
                print("Problem is unbounded.")
                return None
            ratios = np.array([new_A[i][-1] / new_A[i][key_column] for i in possible_rows])
            ratios = np.where(ratios > 0, ratios, np.inf)   
            min_ratio_idx = np.argmin(ratios)
            key_row = possible_rows[min_ratio_idx]
            print(f"Possible rows: {possible_rows}")
            print(f"Ratios: {ratios}")
            print(f"Leaving variable row (key row): {key_row}")
            pivot_element = new_A[key_row][key_column]
            print(f"Pivot element: {pivot_element}")
            if abs(pivot_element) < 1e-10:
                print("Division by zero encountered (pivot is zero).")
                return None
            for j in range(len(new_A[key_row])):
                new_A[key_row][j] /= pivot_element
            for i in range(n_constraints):
                if i != key_row:
                    factor = new_A[i][key_column]
                    for j in range(len(new_A[i])):
                        new_A[i][j] -= factor * new_A[key_row][j]
            old_basic = basic[key_row]
            basic[key_row] = key_column
            cb[key_row] = new_B[key_column]
            print(f"Variable {old_basic} leaves, variable {key_column} enters")
            print(f"Updated basic variables: {basic}")
            print(f"Updated cb_matrix: {cb}")
            zj, cj = update_zj_cj_two_phase_2(new_A,new_B,cb,n_variables,n_constraints,greater_than_count,less_than_count,equal_to_count)
            print(f"Updated Cj: {[round(c, 6) for c in cj]}")
            print("Current tableau:")
            for i, row in enumerate(new_A):
                print(f"Row {i}: {[round(x, 6) for x in row]}")
        print(new_A[-1])
        if(-1 not in cb):
           entering_phase_2 = True
        if(entering_phase_2):
         print(f"\n--- Final Solution (after {iteration} iterations) ---")
         x = [0.0] * n_variables
         for i in range(n_constraints):
          col = basic[i]
          if col < n_variables:
            x[col] = new_A[i][-1]
         bj = [row[-1] for row in new_A]
         optimal_value = np.dot(x, objective_fun)
         print("CB: ",cb)
         print(f"Basic variables: {basic}")
         print(f"Solution: {[round(xi, 6) for xi in x]}")
         print(f"Optimal value: {round(optimal_value, 6)}")
        else:
          print("No solution possible")
    return -1
# objective_fun=[5,-2,3]
# A=[[2,2,-1],[6,5,10],[8,-3,6]]
# B=[2,76,50]
# _type=['>=','<=','<=']

# objective_fun = [2,1]
# A=[[1,2],[1,1],[1,-1],[1,2],[2,-3]]
# B=[10,6,2,1,1]
# _type = ['<=']*5

objective_fun=[2,3,1]
A=[[1,1,1],[2,1,-1],[0,-1,1]]
B=[40,10,10]
_type = ['<=','>=','>=']

# objective_fun=[4,5]
# A=[[2,3],[3,1]]
# B=[6,3]
# _type = ['<=','>=']

# objective_fun=[2,3]
# A=[[0.5,0.25],[1,3],[1,1]]
# B=[4,36,10]
# _type = ['<=','<=','=']


# objective_fun=[-4,-1]
# A=[[1,2],[4,3],[3,1]]
# B=[3,6,3]
# _type = ['<=','>=','=']
result = two_phase_simplex_method_generalized(objective_fun, A, B, 'maximize',_type)

[0, 0, 0, 0, 0, 0, -1, -1]
Initial tableau:
[1, 1, 1, 1, 0, 0, 0, 0, 40]
[2, 1, -1, 0, -1, 0, 1, 0, 10]
[0, -1, 1, 0, 0, -1, 0, 1, 10]

Initial Cj: [np.int64(-2), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0)]
Initial basic variables: [3, 4, 5]
Initial cb_matrix: [0, -1, -1]
Initial zj: [np.int64(-2), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(-1), np.int64(-1)]
--------------------------------------------------

--- Iteration 1 ---
Possible rows: [0, 1]
Ratios: [40.  5.]
Leaving variable row (key row): 1
Pivot element: 2
Variable 4 leaves, variable 0 enters
Updated basic variables: [3, np.int64(0), 5]
Updated cb_matrix: [0, 0, -1]
Updated Cj: [np.float64(0.0), np.float64(1.0), np.float64(-1.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(1.0), np.float64(0.0)]
Current tableau:
Row 0: [0.0, 0.5, 1.5, 1.0, 0.5, 0.0, -0.5, 0.0, 35.0]
Row 1: [1.0, 0.5, -0.5, 0.0, -0.5, 0.0, 0.5, 0.0, 5.0]
Row 2: [